# Imports 

In [ ]:
import numpy as np
import imageio
from matplotlib.pyplot import imshow
import PIL.Image as Image
import cv2


In [ ]:
imageio.plugins.freeimage.download()

In [ ]:
image = Image.open("UE1.png")
arr = np.asarray(image)
print(arr)
imshow(arr)
#arr = cv2.imread("UE1.png", cv2.IMREAD_UNCHANGED).astype(np.float32)

# freeimage lib only supports float32 not float64 arrays
arr = arr.astype("float32")

# # Write to disk
imageio.imwrite('pil.exr', arr)


In [ ]:
# Read created exr from disk
img = imageio.imread('float_img.exr')
np.equal(img,arr)

# Make Dataset Folders

In [64]:
import io
import os
from glob import glob
from PIL import Image, ImageEnhance
import numpy as np
import shutil

os.chdir('E:/')
dir_src = ('E:/Dataset/SuperCaustics/')

Depth =   (dir_src + "/Depth/")
Normal = (dir_src + "/Normal/")
Pic =     (dir_src + "/Picture/")
Outlines = (dir_src + "/Outlines/")
Seg = (dir_src + "/Seg/")

Depth_exr =   (dir_src + "/Depth_exr/")
Normal_exr = (dir_src + "/Normal_exr/")
Seg_exr = (dir_src + "/Seg_exr/")

try:
    os.mkdir(Depth)
    os.mkdir(Normal)
    os.mkdir(Pic)
    os.mkdir(Outlines)
    os.mkdir(Seg)
    os.mkdir(Depth_exr)
    os.mkdir(Normal_exr)
    os.mkdir(Seg_exr)

except OSError:
        print('Directory not created.')



# Move images into their respective folders (Order is important)

In [65]:
counter = 1
for filename in os.listdir(dir_src):

    if counter == 1:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Pic)
            print("moved " + filename)
            counter = counter+1

    elif counter == 2:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Depth) #Depth
            print("moved " + filename)
            counter = counter+1

    elif counter == 3:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Normal) #Normal
            print("moved " + filename)
            counter = counter+1
            
    elif counter == 4:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Outlines) #Outlines
            print("moved " + filename)
            counter = counter+1

    elif counter == 5:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Seg) #Seg
            print("moved " + filename)
            counter = 1


moved HighresScreenshot00000.png
moved HighresScreenshot00001.png
moved HighresScreenshot00002.png
moved HighresScreenshot00003.png
moved HighresScreenshot00004.png
moved HighresScreenshot00005.png
moved HighresScreenshot00006.png
moved HighresScreenshot00007.png
moved HighresScreenshot00008.png
moved HighresScreenshot00009.png
moved HighresScreenshot00010.png
moved HighresScreenshot00011.png
moved HighresScreenshot00012.png
moved HighresScreenshot00013.png
moved HighresScreenshot00014.png
moved HighresScreenshot00015.png
moved HighresScreenshot00016.png
moved HighresScreenshot00017.png
moved HighresScreenshot00018.png
moved HighresScreenshot00019.png


# Rename raw images in each folder to have the same name

In [66]:
i = 0
for filename in os.listdir(Pic):
    os.rename(Pic + '/' + filename, Pic + '/' + str(i) + '.png')
    i = i + 1
    print("renamed " + filename)

j = 0
for filename in os.listdir(Depth):
    os.rename(Depth + '/' + filename, Depth + '/' + str(j) + '.png')
    j = j + 1
    print("renamed " + filename)

k = 0
for filename in os.listdir(Normal):
    os.rename(Normal + '/' + filename, Normal + '/' + str(k) + '.png')
    k = k + 1
    print("renamed " + filename)
    
l = 0
for filename in os.listdir(Outlines):
    os.rename(Outlines + '/' + filename, Outlines + '/' + str(l) + '.png')
    l = l + 1
    print("renamed " + filename)
    
m = 0
for filename in os.listdir(Seg):
    os.rename(Seg + '/' + filename, Seg + '/' + str(m) + '.png')
    m = m + 1
    print("renamed " + filename)


renamed HighresScreenshot00000.png
renamed HighresScreenshot00005.png
renamed HighresScreenshot00010.png
renamed HighresScreenshot00015.png
renamed HighresScreenshot00001.png
renamed HighresScreenshot00006.png
renamed HighresScreenshot00011.png
renamed HighresScreenshot00016.png
renamed HighresScreenshot00002.png
renamed HighresScreenshot00007.png
renamed HighresScreenshot00012.png
renamed HighresScreenshot00017.png
renamed HighresScreenshot00003.png
renamed HighresScreenshot00008.png
renamed HighresScreenshot00013.png
renamed HighresScreenshot00018.png
renamed HighresScreenshot00004.png
renamed HighresScreenshot00009.png
renamed HighresScreenshot00014.png
renamed HighresScreenshot00019.png


# Converting To EXR for Cleargrasp

In [75]:
for filename in os.listdir(Depth):
    address = Depth + filename
    print(address)
    image = Image.open(address)
    arr = np.asarray(image)
    arr = arr.astype("float32")
    imageio.imwrite(address.replace('.png','.exr'), arr)
    

E:/Dataset/SuperCaustics//Depth/0.png
E:/Dataset/SuperCaustics//Depth/1.png
E:/Dataset/SuperCaustics//Depth/2.png
E:/Dataset/SuperCaustics//Depth/3.png
